In [2]:

import tkinter as tk
from tkinter import messagebox, simpledialog
import os

ARCHIVO_CONTACTOS = "friendsContact.txt"  # nombre del archivo donde guardamos todo
COLOR_FONDO = "#f0f0f0"  # Gris claro para la ventana principal
COLOR_BOTON = "#4CAF50"   # Verde para los botones principales
COLOR_TEXTO = "#333333"    # Gris oscuro para textos
COLOR_PELIGRO = "#d9534f"  # Rojo para botones de cerrar


def asegurar_archivo():
    """Crea el archivo si no existe para evitar errores al abrir."""
    archivo_existe = os.path.exists(ARCHIVO_CONTACTOS)
    if not archivo_existe:
        archivo_nuevo = open(ARCHIVO_CONTACTOS, "a", encoding="utf-8")
        archivo_nuevo.close()


def leer_lineas():
    """Lee todas las líneas del archivo y las devuelve como lista."""
    asegurar_archivo()
    with open(ARCHIVO_CONTACTOS, "r", encoding="utf-8") as f:
        todas = f.readlines()
    lista_sin_saltos = []
    for linea in todas:
        linea_limpia = linea.strip()
        if linea_limpia != "":
            lista_sin_saltos.append(linea_limpia)
    return lista_sin_saltos


def escribir_lineas(lineas):
    """Sobrescribe el archivo con la lista de líneas proporcionada."""
    with open(ARCHIVO_CONTACTOS, "w", encoding="utf-8") as f:
        for linea in lineas:
            f.write(linea + "\n")


def crear_contacto(nombre, numero):
    """Guarda un nuevo contacto o actualiza si el nombre ya existe."""
    nombre = nombre.strip()
    numero = numero.strip()

    if nombre == "" or numero == "":
        return "Debe ingresar nombre y número"

    contactos_actuales = leer_lineas()

    contactos_actualizados = []  
    encontrado = False          

    for contacto in contactos_actuales:
        partes = contacto.split(",")  
        nombre_guardado = partes[0]
        numero_guardado = partes[1]

        if nombre_guardado == nombre:
           
            contactos_actualizados.append(f"{nombre},{numero}")
            encontrado = True
        else:
          
            contactos_actualizados.append(contacto)

    if not encontrado:

        contactos_actualizados.append(f"{nombre},{numero}")

    escribir_lineas(contactos_actualizados)
    return "✓ Contacto guardado"


def leer_contactos(nombre_busqueda=None):
    """Devuelve lista de contactos o solo uno si se pasa nombre_busqueda."""
    contactos = leer_lineas()

    if nombre_busqueda is None or nombre_busqueda.strip() == "":
        return contactos

    resultado = []  
    for contacto in contactos:
        partes = contacto.split(",")
        nombre_guardado = partes[0]
        if nombre_guardado == nombre_busqueda:
            resultado.append(contacto)
            break 
    return resultado


def actualizar_contacto(nombre, nuevo_numero):
    """Actualiza el número de un contacto existente."""
    contactos = leer_lineas()
    nuevo_contenido = []
    actualizado = False

    for contacto in contactos:
        partes = contacto.split(",")
        nombre_guardado = partes[0]
        numero_guardado = partes[1]

        if nombre_guardado == nombre:
        
            nuevo_contenido.append(f"{nombre},{nuevo_numero.strip()}")
            actualizado = True
        else:
            nuevo_contenido.append(contacto)

    if not actualizado:
        return "Contacto no encontrado"

    escribir_lineas(nuevo_contenido)
    return "✓ Contacto actualizado"


def eliminar_contacto(nombre):
    """Elimina el contacto con el nombre indicado."""
    contactos = leer_lineas()
    nuevo_contenido = []
    eliminado = False

    for contacto in contactos:
        partes = contacto.split(",")
        nombre_guardado = partes[0]
        if nombre_guardado == nombre:
            eliminado = True  
        else:
            nuevo_contenido.append(contacto)

    if not eliminado:
        return "Contacto no encontrado"

    escribir_lineas(nuevo_contenido)
    return "✓ Contacto eliminado"

class VentanaContactos:
    def __init__(self, root):
        
        self.root = root
        self.root.title("Gestor de Contactos - Versión Estudiante")
        self.root.configure(bg=COLOR_FONDO)

        etiqueta_nombre = tk.Label(root, text="Nombre:", bg=COLOR_FONDO, fg=COLOR_TEXTO)
        etiqueta_nombre.grid(row=0, column=0, padx=10, pady=5)
        self.entrada_nombre = tk.Entry(root, width=25)
        self.entrada_nombre.grid(row=0, column=1, padx=10, pady=5)

        etiqueta_numero = tk.Label(root, text="Número:", bg=COLOR_FONDO, fg=COLOR_TEXTO)
        etiqueta_numero.grid(row=1, column=0, padx=10, pady=5)
        self.entrada_numero = tk.Entry(root, width=25)
        self.entrada_numero.grid(row=1, column=1, padx=10, pady=5)

        config_boton = {"bg": COLOR_BOTON, "fg": "white", "width": 12, "height": 1}

        boton_crear = tk.Button(root, text="CREAR", command=self.crear, **config_boton)
        boton_crear.grid(row=2, column=0, pady=8)

        boton_leer = tk.Button(root, text="LEER", command=self.leer, **config_boton)
        boton_leer.grid(row=2, column=1, pady=8)

        boton_actualizar = tk.Button(root, text="ACTUALIZAR", command=self.actualizar, **config_boton)
        boton_actualizar.grid(row=3, column=0, pady=8)

        boton_eliminar = tk.Button(root, text="ELIMINAR", command=self.eliminar, **config_boton)
        boton_eliminar.grid(row=3, column=1, pady=8)

        # 4.3) Área de texto para mostrar resultados (solo para listas)
        self.area_texto = tk.Text(root, width=44, height=12, bg="white", fg=COLOR_TEXTO, state="disabled")
        self.area_texto.grid(row=4, column=0, columnspan=2, padx=10, pady=10)
        self.mostrar_mensaje("➤ Lista de contactos aparecerá aquí")

    def crear(self):
    
        nombre_ingresado = self.entrada_nombre.get()
        numero_ingresado = self.entrada_numero.get()

        resultado = crear_contacto(nombre_ingresado, numero_ingresado)
        messagebox.showinfo("Resultado", resultado)

        self.entrada_nombre.delete(0, tk.END)
        self.entrada_numero.delete(0, tk.END)
        self.leer()  

    def leer(self):
        nombre_busqueda = self.entrada_nombre.get().strip()

        if nombre_busqueda != "":
            lista_resultado = leer_contactos(nombre_busqueda)
            if len(lista_resultado) == 0:
                messagebox.showinfo("Sin resultados", f"No se encontró el contacto '{nombre_busqueda}'")
            else:
                contacto = lista_resultado[0]
                partes = contacto.split(",")
                nombre = partes[0]
                numero = partes[1]

                popup = tk.Toplevel(self.root)
                popup.title("Contacto encontrado")
                popup.configure(bg="white")

                etiqueta1 = tk.Label(popup, text=f"Nombre: {nombre}", bg="white", font=("Arial", 12, "bold"))
                etiqueta1.pack(padx=20, pady=(20, 10))
                etiqueta2 = tk.Label(popup, text=f"Número: {numero}", bg="white", font=("Arial", 12))
                etiqueta2.pack(padx=20, pady=(0, 20))

                boton_cerrar = tk.Button(popup, text="Cerrar", bg=COLOR_PELIGRO, fg="white", width=10, command=popup.destroy)
                boton_cerrar.pack(pady=(0, 20))

                popup.transient(self.root)
                popup.grab_set()
                self.root.wait_window(popup)
        else:
            
            contactos = leer_contactos()
            if len(contactos) == 0:
                self.mostrar_mensaje("➤ No hay contactos registrados")
            else:
                texto_mostrar = "📚 LISTA DE CONTACTOS:\n\n"
                for c in contactos:
                    partes = c.split(",")
                    nombre = partes[0]
                    numero = partes[1]
                    texto_mostrar += f"• {nombre}: {numero}\n"
                self.mostrar_mensaje(texto_mostrar)

    def actualizar(self):
        nombre_ingresado = self.entrada_nombre.get().strip()
        if nombre_ingresado == "":
            messagebox.showerror("Error", "Ingrese nombre a actualizar")
            return

        nuevo_numero = simpledialog.askstring("Actualizar", f"Nuevo número para {nombre_ingresado}:")
        if nuevo_numero is not None and nuevo_numero.strip() != "":
            resultado = actualizar_contacto(nombre_ingresado, nuevo_numero)
            messagebox.showinfo("Resultado", resultado)
            self.leer()

    def eliminar(self):
        nombre_ingresado = self.entrada_nombre.get().strip()
        if nombre_ingresado == "":
            messagebox.showerror("Error", "Ingrese nombre a eliminar")
            return
        resultado = eliminar_contacto(nombre_ingresado)
        messagebox.showinfo("Resultado", resultado)
        self.leer()

    def mostrar_mensaje(self, mensaje):
        self.area_texto.configure(state="normal")
        self.area_texto.delete("1.0", tk.END)
        self.area_texto.insert(tk.END, mensaje)
        self.area_texto.configure(state="disabled")


if __name__ == "__main__":
    ventana_principal = tk.Tk()
    app = VentanaContactos(ventana_principal)
    ventana_principal.mainloop()
